In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
Train=pd.read_csv("/kaggle/input/tabular-playground-series-sep-2021/train.csv")
Test=pd.read_csv("/kaggle/input/tabular-playground-series-sep-2021/test.csv")


In [ ]:
Test1=pd.read_csv("/kaggle/input/tabular-playground-series-sep-2021/sample_solution.csv")

In [ ]:
print(Train.shape)
print(Test.shape)
print(Train.dtypes)
print(Test.dtypes)
print(Train.dtypes.unique())
print(Test.dtypes.unique())


In [ ]:
print(Train.isna().sum())
print(Test.isna().sum())

In [ ]:
num_cols=Train.select_dtypes('number').columns
obj_cols=Train.select_dtypes('object').columns
print(num_cols)
print(obj_cols)

In [ ]:
y=Train['claim']
Train=Train.drop(['claim'], axis=1)
Train.shape


In [ ]:
#Fill missing values
from sklearn.impute import SimpleImputer

# Imputation
my_imputer = SimpleImputer()
imputed_train = pd.DataFrame(my_imputer.fit_transform(Train))
imputed_test = pd.DataFrame(my_imputer.transform(Test))

# Imputation removed column names; put them back
imputed_train.columns = Train.columns
imputed_test.columns = Test.columns

In [ ]:
#standardization
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
standard1 = scaler.fit_transform(imputed_train)
standard2 = scaler.transform(imputed_test)

#normalization

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
normal1 = scaler.fit_transform(standard1)
normal2 = scaler.transform(standard2)

Train = pd.DataFrame(normal1)
print(Train.shape)
Test = pd.DataFrame(normal2)

#useful_features= [c for c in df.columns if c not in ("id")]

In [ ]:
"""
# Use PCA
from sklearn.decomposition import PCA
 
pca = PCA(n_components = 90)
 
Train = pca.fit_transform(normal1)
Test = pca.transform(normal2)

Train = pd.DataFrame(Train)
print(Train.shape)
Test = pd.DataFrame(Test)
print(Test.shape)

"""

In [ ]:
"""
What Techniques to Use When
Generally, k-fold cross validation is the gold-standard for evaluating the performance of a machine learning algorithm on unseen data with k set to 3, 5, or 10.
Use stratified cross validation to enforce class distributions when there are a large number of classes or an imbalance in instances for each class.
Using a train/test split is good for speed when using a slow algorithm and produces performance estimates with lower bias when using large datasets.
The best advice is to experiment and find a technique for your problem that is fast and produces reasonable estimates of performance that you can use to make decisions.

If in doubt, use 10-fold cross validation for regression problems and stratified 10-fold cross validation on classification problems.

"""

In [ ]:
"""

#Using stratified K-fold split along with XGBClassifier

from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.metrics import roc_auc_score
model= XGBClassifier(n_estimators = 2000, use_label_encoder= False,objective='binary:logistic', max_depth=8, learning_rate=0.01, alpha=0,tree_method='gpu_hist',gpu_id=0,predictor="gpu_predictor", random_state =20)
result=cross_val_score(estimator=model,X=Train,y=y,scoring='roc_auc',cv=5) #https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html
result

"""

In [ ]:
"""
#Using train,test split along with XGBClassifier

from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(Train, y, test_size = 0.3, random_state = 0)
print(X_train.shape)
print(X_valid.shape)
print(y_train.shape)
print(y_valid.shape)

from xgboost.sklearn import XGBClassifier
from sklearn.metrics import roc_auc_score

params= {'n_estimators' : 1500, 
         'use_label_encoder' : 'False',
         'objective' :'binary:logistic', 
         'max_depth' : 6, 
         'learning_rate' : 0.02, 
         'alpha' : 10,
         'tree_method' :'gpu_hist',
         'gpu_id' : 0,
         'predictor' :"gpu_predictor", 
         'random_state' : 20}

model= XGBClassifier(**params)
model.fit(X_train, y_train)
y_pred = model.predict(X_valid)

#from sklearn.metrics import accuracy_score
print('XGBoost model accuracy score: {0:0.4f}'. format(roc_auc_score(y_valid, y_pred)))
"""



In [ ]:
"""
#XGB classfier with hyperparameter tuning using Hyperopt 

from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(Train, y, test_size = 0.3, random_state = 0)
print(X_train.shape)
print(X_valid.shape)
print(y_train.shape)
print(y_valid.shape)

from hyperopt import hp
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import roc_auc_score

space={'max_depth': hp.quniform("max_depth", 3, 18, 1),
       'learning_rate' : hp.quniform("learning_rate", 0, 1, 0.01), 
        'alpha' : hp.uniform("alpha", 1, 9),
        'gamma': hp.uniform ('gamma', 1,9),
        'reg_alpha' : hp.quniform('reg_alpha', 40,180,1),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'n_estimators': hp.choice('n_estimators', range(50,1000)),
        'seed': 0
    }

def objective(space):
    
    clf=XGBClassifier(
                    n_estimators =space['n_estimators'], use_label_encoder= False, max_depth = int(space['max_depth']), 
                    gamma = space['gamma'],reg_alpha = int(space['reg_alpha']), 
                    min_child_weight=int(space['min_child_weight']),
                    tree_method='gpu_hist',gpu_id=0,predictor="gpu_predictor", random_state =20,
                    colsample_bytree=int(space['colsample_bytree']))
    
    evaluation = [( X_train, y_train), ( X_valid, y_valid)]
    
    clf.fit(X_train, y_train,
            eval_set=evaluation, eval_metric="auc",
            early_stopping_rounds=10,verbose=False)
    

    pred = clf.predict(X_valid)
    accuracy = roc_auc_score(y_valid, pred>0.5)
    print ("SCORE:", accuracy)
    return {'loss': -accuracy, 'status': STATUS_OK }

trials = Trials()

best_hyperparams = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 100,
                        trials = trials)

print("The best hyperparameters are : ","\n")
print(best_hyperparams)


params= {'n_estimators' : 184, 
         'use_label_encoder' : 'False',
         'objective' :'binary:logistic', 
         'learning_rate': 0.33,
         'max_depth' : 18, 
         'min_child_weight' : 8,
         'alpha': 2.7457662489762638,
         'reg_alpha' :41,
         'reg_lambda': 0.5127805406197737,
         'gamma': 1.2189238435974041,
         'colsample_bytree': 0.5759927696082564,
         'tree_method' :'gpu_hist',
         'gpu_id' : 0,
         'predictor' :"gpu_predictor", 
         'random_state' : 20}

model= XGBClassifier(**params)
model.fit(X_train, y_train)
y_pred = model.predict(X_valid)

#from sklearn.metrics import accuracy_score
print('XGBoost model accuracy score: {0:0.4f}'. format(roc_auc_score(y_valid, y_pred)))

Predict1 = model.predict(Test)
output = pd.DataFrame({'id': Test1.id,
                       'claim': Predict1})
output.to_csv('submission.csv', index=False)


"""

In [ ]:
#XGB classfier with hyperparameter tuning using Hyperopt, with cross validation 

from hyperopt import hp
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
import xgboost as xgb

space={'max_depth': hp.quniform("max_depth", 3, 18, 1),
       'learning_rate' : hp.quniform("learning_rate", 0, 1, 0.01), 
        'alpha' : hp.uniform("alpha", 1, 9),
        'gamma': hp.uniform ('gamma', 1,9),
        'reg_alpha' : hp.quniform('reg_alpha', 40,180,1),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'n_estimators': hp.choice('n_estimators', range(100,2000)),
    }

def objective(space):
    
    clf=XGBClassifier( 
                    use_label_encoder= False, 
                    max_depth = int(space['max_depth']),
                    learning_rate =space['learning_rate'],
                    alpha = space['alpha'],
                    gamma = space['gamma'],
                    reg_alpha = int(space['reg_alpha']),
                    reg_lambda = int(space['reg_lambda']),
                    colsample_bytree=int(space['colsample_bytree']),
                    min_child_weight=int(space['min_child_weight']),
                    n_estimators =space['n_estimators'],
                    seed=0,
                    objective='binary:logistic',
                    tree_method='gpu_hist',
                    gpu_id=0,
                    predictor="gpu_predictor", 
                    )
    
    
    clf.fit(Train, y)

    # Applying k-Fold Cross Validation
    from sklearn.model_selection import cross_val_score
    accuracies = cross_val_score(estimator = clf, X = Train, y = y,scoring='roc_auc', cv = 5)
    CrossValMean = accuracies.mean()

    print("CrossValMean:", CrossValMean)

    return{'loss':1-CrossValMean, 'status': STATUS_OK }
    
    
    #result=cross_val_score(estimator=clf,X=Train,y=y,scoring='roc_auc',cv=5) #https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html



In [ ]:
trials = Trials()

best_hyperparams = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 5,
                        trials = trials)

In [ ]:
print("The best hyperparameters are : ","\n")
print(best_hyperparams)

In [ ]:
# Fitting XGBoost to the Training set
from xgboost import XGBClassifier

#warnings.filterwarnings(action='ignore', category=DeprecationWarning)
clf = XGBClassifier(n_estimators = best_hyperparams['n_estimators'],
                            max_depth = int(best_hyperparams['max_depth']),
                            learning_rate = best_hyperparams['learning_rate'],
                            gamma = best_hyperparams['gamma'],
                            min_child_weight = best_hyperparams['min_child_weight'],
                            colsample_bytree = best_hyperparams['colsample_bytree'],
                            alpha = best_hyperparams['alpha'],
                            reg_alpha = best_hyperparams['reg_alpha'],
                            reg_lambda = best_hyperparams['reg_lambda'],
                            seed = 0,
                            objective='binary:logistic',
                            tree_method='gpu_hist',
                            gpu_id=0,
                            predictor="gpu_predictor"
                            )


clf.fit(Train, y)

# Applying k-Fold Cross Validation
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = clf, X = Train, y = y, cv = 5)
print(accuracies)
CrossValMean = accuracies.mean()
print("Final CrossValMean: ", CrossValMean)

In [ ]:
Predict1 = clf.predict(Test)
output = pd.DataFrame({'id': Test1.id,
                       'claim': Predict1})
output.to_csv('submission.csv', index=False)